In [0]:
SELECT
  count(*)
FROM
  users.guy_livni.incoming_radiology_reports;

SELECT
  *
FROM
  users.guy_livni.incoming_radiology_reports;

/* Scenario: We have a stream of raw HL7/Radiology report text coming into the 'incoming_reports' table.
  Goal: Extract the finding and urgency to route to the correct care pathway.
*/
SELECT
  report_id,
  raw_text,
  /* 1. Extract structured data from free text */
  ai_extract(
    raw_text,
    array('primary_finding', 'urgency_level', 'affected_organ', 'is_contrast_used')
  ) as ai_structured_data,
  /* 2. Classify for routing logic */
  ai_classify(
    raw_text,
    array('Critical - Immediate Notification', 'Routine - Standard Queue', 'Follow-up Required')
  ) as triage_category
FROM
  incoming_radiology_reports;

CREATE OR REPLACE TABLE users.guy_livni.incoming_radiology_reports_findings as
SELECT
  report_id,
  patient_id,
  ingestion_date,
  referring_physician,
  raw_text,
  /* 1. Extract structured data from free text */
  ai_extract(
    raw_text,
    array('primary_finding', 'urgency_level', 'affected_organ', 'is_contrast_used')
  ) as ai_structured_data,
  /* 2. Classify for routing logic */
  ai_classify(
    raw_text,
    array('Critical - Immediate Notification', 'Routine - Standard Queue', 'Follow-up Required')
  ) as triage_category,
  ground_truth_critical
FROM
  incoming_radiology_reports;

select
  count(*)
from
  users.guy_livni.incoming_radiology_reports_findings;

select
  *
from
  users.guy_livni.incoming_radiology_reports_findings
where
  triage_category = 'Critical - Immediate Notification';

/* Analyze sentiment to gauge urgency/concern level in radiologist language */
SELECT 
  report_id,
  raw_text,
  triage_category,
  ai_structured_data.primary_finding,
  ai_analyze_sentiment(raw_text) as sentiment_analysis :param_1
FROM users.guy_livni.incoming_radiology_reports_findings
ORDER BY report_id;

SELECT
  *
from
  users.guy_livni.incoming_radiology_reports_findings
;

/* Triage category distribution by hour */
SELECT
  date_trunc('hour', ingestion_date) as report_hour,
  triage_category,
  count(*) as report_count,
  round(count(*) * 100.0 / sum(count(*)) OVER (PARTITION BY date_trunc('hour', ingestion_date)), 2) as percentage_of_hour
FROM
  users.guy_livni.incoming_radiology_reports_findings
GROUP BY
  date_trunc('hour', ingestion_date),
  triage_category
ORDER BY
  report_hour,
  triage_category;